In [1]:
import os

In [2]:
%pwd

'd:\\KIDNEY_DISEASE_CLASSIFICATION-Deep_Learning_Project\\research'

In [3]:
#os.chdir("..")
os.chdir('d:/KIDNEY_DISEASE_CLASSIFICATION-Deep_Learning_Project')

In [4]:
%pwd

'd:\\KIDNEY_DISEASE_CLASSIFICATION-Deep_Learning_Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

#getting root na dtrained modelpath from config yaml file
#then get updated model and training data get from artifacts
#params from the params.yaml file

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

configuration Management

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

#inside it write get modeltraiing related configuration
#here giving all the path
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        #here pass updated base model path from congif.yaml
        #also give path of kidney ct scan inside artifacts inside data ingestation
        
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

##update the component

In [8]:

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf 
import time

initialize training

In [9]:
#define training class and again define training configuration get from configmanager
#then getbasemodel load from artifacts


class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    #it will generate thr training and validation data 
    #here not separately doing traintest split here keras inbuild train_valid_generator
    #creating training dataset and testing dataset
    def train_valid_generator(self):

        #here doing some data augmentation like preform scaling operation
        #get from keras toget trainvalid generator
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    #this is the training function model will start training
    #firstly calculate stepsperepoch and validation_steps 
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        #training model
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

now define the pipeline

In [10]:
#initialize configmanager and call get_training_cofig
#then this training_config call inside training initialize
# then call get base model it load the model #then trainvlaid genator and then train
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-06-06 13:42:36,678: INFO: common: yaml file: d:\KIDNEY_DISEASE_CLASSIFICATION-Deep_Learning_Project\config\config.yaml loaded successfully]
[2024-06-06 13:42:36,689: INFO: common: yaml file: d:\KIDNEY_DISEASE_CLASSIFICATION-Deep_Learning_Project\params.yaml loaded successfully]
[2024-06-06 13:42:36,692: INFO: common: created directory at: artifacts]
[2024-06-06 13:42:36,694: INFO: common: created directory at: artifacts\training]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
23/23 [==============================] - 84s 4s/step - loss: 12.4105 - accuracy: 0.5393 - val_loss: 7.2387 - val_accuracy: 0.4375


So training is done and this this metricds we get
for good model train need 200-300 epoch